### Imports

In [1]:
import requests

from arcgis.gis import GIS
from arcgis.features import FeatureLayer

import numpy as np
import pandas as pd

### Connecting to ArcGIS

In [2]:
# Login for the notebook running in AGOL
#gis = GIS("home")

# Login for the notebook running in Pro locally...
gis = GIS("pro")

### Variables
Currently the production layer (which is already powering the very very pretty Dashboard) is commented out and I'm grabbing a test layer instead (next block down). Will un-comment the operational layer when I'm 100% sure things aren't blowing up anymore.

In [3]:

# The Item ID of the service containing both the geometry layers and the dashboard layer
# dd is "Disaster Declarations"
dd_id = "d37c3c2a6f1c4586baad82828bfc3c59"

# Get the item at this item ID
dd_item = gis.content.get(dd_id)

# Item ID 1 is the input layer used for getting geometries
geometries_layer = dd_item.layers[1]

print(geometries_layer)

# Item ID 0 is the output layer displayed in the dashboard
# COMMENTING OUT WHILE TESTING
#dashboard_layer = dd_item.layers[0]

#print(dashboard_layer)

<FeatureLayer url:"https://services9.arcgis.com/GDVaV4SDJDDBT8gi/arcgis/rest/services/Disaster_Declarations_Summaries_v2/FeatureServer/1">


### Get the Dashboard layer FOR TESTING ONLY

In [4]:
# ID of the item including Disaster Declarations Summaries subset FOR TESTING ONLY,
# replaces "dashboard_layer" above through duration of testing
test_id = "edb716da51bc4f7882d13d425ad08fd2"

test_item = gis.content.get(test_id)

dashboard_layer = test_item.layers[0]

print(dashboard_layer)

<FeatureLayer url:"https://services9.arcgis.com/GDVaV4SDJDDBT8gi/arcgis/rest/services/DisasterDeclarations_forTesting_2025only/FeatureServer/0">


### Connect to OpenFEMA API and get Disaster Declarations Summaries
* Right right right; I forgot that the API by default only returns 1000 records. I shouldn't really NEED more records than that, since the script is going to be run once per day. One thousand records should be MORE than enough. But, I now realize I need to do a little footwork to make sure I am just getting the 1000 _most recent_ records...

* Okay added a sort order to the api call to only get the most recent records by declarationDate! That should do it...

In [5]:
# Within the API URL, filter the records to return only fyDeclared to 2013 or newer.
# US Census GDBs only go back to 2013; before that it's shapefiles only
# and I refuse to touch shapefiles, at least for the scope of this project.

api_url = r"https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$filter=fyDeclared ge 2013&$orderby=declarationDate desc"

# Plug in the URL and capture the response obj
response = requests.get(api_url)

# Convert response to JSON
data = response.json()

# Okay so after a little digging I really only need the following (leave out the metadata)
summaries_df = pd.DataFrame(data["DisasterDeclarationsSummaries"])

summaries_df.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id
0,FM-5612-CA,5612,CA,FM,2025-09-03T00:00:00.000Z,2025,Fire,2-7 FIRE,False,False,...,99009,Calaveras (County),25121,None,2025090301,9,R,2025-09-03T18:41:07.857Z,d017531813b75fc753371c26b246931d48de651e,28a1ba9f-d914-4024-9e75-4a66b5bba092
1,FM-5611-MT,5611,MT,FM,2025-08-26T00:00:00.000Z,2025,Fire,WINDY ROCK FIRE,False,False,...,99077,Powell (County),25119,None,2025082701,8,R,2025-08-28T18:01:23.160Z,29e175a73b969da6864182e703e3cb3f8d0bb32d,41329e57-2046-4196-a63d-902f3e7c923c
2,FM-5610-OR,5610,OR,FM,2025-08-23T00:00:00.000Z,2025,Fire,FLAT FIRE,False,False,...,99017,Deschutes (County),25117,None,2025082301,10,R,2025-08-25T18:21:58.453Z,c4a190d030807595da90813aabc6ad2175917668,df7cb24f-8e5a-4c1e-923e-4c75c9ec4581
3,FM-5610-OR,5610,OR,FM,2025-08-23T00:00:00.000Z,2025,Fire,FLAT FIRE,False,False,...,99031,Jefferson (County),25117,None,2025082301,10,R,2025-08-25T18:21:58.453Z,8b07b29243bdbba511790332bd3fa9cca0fe33fd,f0604c05-113b-449e-8e4a-f3b5076af546
4,FM-5609-HI,5609,HI,FM,2025-08-19T00:00:00.000Z,2025,Fire,KUNIA ROAD FIRE,False,False,...,99003,Honolulu (County),25114,None,2025082001,9,R,2025-08-21T18:22:16.374Z,731df26a647e5a0338177f445bab7a23b6f8d6ed,ffab7fa0-2e69-428d-b4d3-da95ca352c03


### Convert pseudo-date columns to actual date columns
I checked all the fips / code fields to ensure they're object / string type (I deleted that block while tidying up), so the offending fields remaining are the pseudo-date fields

In [6]:
date_columns = ["declarationDate", "incidentBeginDate", "incidentEndDate", "disasterCloseoutDate", "lastIAFilingDate", "lastRefresh"]

for dc in date_columns:
    summaries_df[dc] = pd.to_datetime(summaries_df[dc], errors="coerce")

### Add full FIPS Code (counties) and full Tribal Code columns

In [7]:
# Add to my df the fields I will need for comparison
summaries_df["fipsFullCode"] = summaries_df["fipsStateCode"] + summaries_df["fipsCountyCode"]
summaries_df["fipsTribalCode"] = summaries_df["fipsStateCode"] + summaries_df["placeCode"]

print(summaries_df["fipsFullCode"].head())
print(summaries_df["fipsTribalCode"].head())

0    06009
1    30077
2    41017
3    41031
4    15003
Name: fipsFullCode, dtype: object
0    0699009
1    3099077
2    4199017
3    4199031
4    1599003
Name: fipsTribalCode, dtype: object


* Considering how I'm going to get the data from the API in shape for the comparison etc. I should just add the two additional columns I added manually for the dashboard I made first, COVID and Entity. After I add and calculate them, the comparisons will all be much easier, because I can just reference those fields for processing the data in chunks (i.e. step 1 process statewide, step 2 process counties, step 3 process tribal)

### Add & calc "COVID19" field

In [8]:
# Good lord I can't remember how to calculate any of these fields with pandas... 🤣😭
# Anyway the first one I need to calc is the COVID column, simple yes/no

summaries_df["COVID19"] = np.where(summaries_df["declarationTitle"].str.contains("COVID-19"), "Show only COVID-19", "Show only non-COVID-19")


### Add & calc "Entities" field

In [9]:
# For my next trick I'll use np.select instead of np.where since to code new Entity column
# I have three possible values not just 2 / yes no / on off

entity_conditions = [
    summaries_df["designatedArea"] == "Statewide",
    (summaries_df["designatedArea"] != "Statewide") & (summaries_df["fipsCountyCode"] == "000"),
    (summaries_df["designatedArea"] != "Statewide") & (summaries_df["fipsCountyCode"] != "000")
]

entity_values = ["State or Equivalent", "Tribal Area or Equivalent", "County or Equivalent"]

summaries_df["Entity"] = np.select(entity_conditions, entity_values)

### Groupby "femaDeclarationString", get temporal range (prelim analysis; to delete)

In [10]:
analyze = summaries_df.groupby("femaDeclarationString")["declarationDate"].agg(["min", "max"])

analyze["range"] = (analyze["max"] - analyze["min"]).dt.total_seconds() / 3600

analyze["range"].max()

0.0

Well that's encouraging (no really I'm not even being sarcastic). I get a whole column of big fat zeros for the declaredDate ranges of all the fema Dec Strings. Honestly...that means I should be fine just proceeding with my simple it's-there-or-not check. I should probably also commit.

OK getting near end-of-day. But now I know I can go forward with a comparison so maybe I'll start that part...

### Let's just re-do the bottom four blocks with a one-liner now that ChatGPT is helping jog my memory from DSTP XD
Can you believe I forgot about isin?? And MERGE??

In [11]:
dashboard_sdf = dashboard_layer.query().sdf

In [12]:
adds_df = summaries_df[~summaries_df["femaDeclarationString"].isin(dashboard_sdf["femaDeclarationString"])]

adds_df

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id,fipsFullCode,fipsTribalCode,COVID19,Entity
0,FM-5612-CA,5612,CA,FM,2025-09-03 00:00:00+00:00,2025,Fire,2-7 FIRE,False,False,...,2025090301,9,R,2025-09-03 18:41:07.857000+00:00,d017531813b75fc753371c26b246931d48de651e,28a1ba9f-d914-4024-9e75-4a66b5bba092,06009,0699009,Show only non-COVID-19,County or Equivalent
1,FM-5611-MT,5611,MT,FM,2025-08-26 00:00:00+00:00,2025,Fire,WINDY ROCK FIRE,False,False,...,2025082701,8,R,2025-08-28 18:01:23.160000+00:00,29e175a73b969da6864182e703e3cb3f8d0bb32d,41329e57-2046-4196-a63d-902f3e7c923c,30077,3099077,Show only non-COVID-19,County or Equivalent
2,FM-5610-OR,5610,OR,FM,2025-08-23 00:00:00+00:00,2025,Fire,FLAT FIRE,False,False,...,2025082301,10,R,2025-08-25 18:21:58.453000+00:00,c4a190d030807595da90813aabc6ad2175917668,df7cb24f-8e5a-4c1e-923e-4c75c9ec4581,41017,4199017,Show only non-COVID-19,County or Equivalent
3,FM-5610-OR,5610,OR,FM,2025-08-23 00:00:00+00:00,2025,Fire,FLAT FIRE,False,False,...,2025082301,10,R,2025-08-25 18:21:58.453000+00:00,8b07b29243bdbba511790332bd3fa9cca0fe33fd,f0604c05-113b-449e-8e4a-f3b5076af546,41031,4199031,Show only non-COVID-19,County or Equivalent


### Good Morning 9/5
I don't think it's tricky that tribal and county original Summary observations with the same Dec string are treated separately. We'll (I'm pretty sure?) be processing all three entity types separately anyway, so there won't be a need to make e.g. a multi-column key or any shenanigans like that.

What do we want to accomplish this fine Friday?
* I need to remove the fields I don't need from the adds_df and then drop dups, effectively dissolving that dataframe.
* I guess somewhere along the way I ought to convert it to a spatial data frame; will probably make adding the rows to the dashboard layer easier.
* BEFORE I do any of the above, I need to use the appropriate field (depending on entity) and grab the correct geometries to dissolve for that Dec string.
* Actually I guess I don't need to do it before; I can do it after; I already have the dfs I need to work with. But procedurally it should happen before.
* What data structure for that? I mean...dict with Dec string as key and list of geom objects as values? What's not to love about that? Gives me warm fuzzies just thinking about it.

If we got even a fraction of the above done today I'd be happy.

### Build dict of primary/foreign keys
Okay now that I have a little more of the script fleshed out, tidying this up and re-doing a few comments here. Main thing I have to remember for further down the script is that THE KEYS ARE THE FIELDS IN THE GEOMETRIES.

In [13]:
# I believe I will need three different queries for my geometries layer
# I'll need to query to get state geometries, county geometries and tribal area geometries:

# Get the states on this field:
# (States we'll check first; )
state_field = "State_FIPS"

# Get the counties on this field:
county_field = "Full_FIPS"

# Crappily, in assembling the original dashboard layer I realized that
# the Declaration data may match ANY ONE of these tribal fields, so I need to check all three:
tribal_field1 = "AIANNHFP1"
tribal_field2 = "AIANNHFP2"
tribal_field3 = "AIANNHFP3"

# Just updated the values below to reflect the fields I've added to the Summaries df; they should be GTG now
key_fields_dict = {
    "State_FIPS": "fipsStateCode",
    "Full_FIPS": "fipsFullCode",
    "AIANNHFP1": "fipsTribalCode",
    "AIANNHFP2": "fipsTribalCode", # No idea why there are three of these...or why the Summaries sometimes use 2 and 3...
    "AIANNHFP3": "fipsTribalCode" # What a pain in the @$$...
}

### Get geometries associated with each Dec string for all three entity types
Since this has to be done technically 5 times (there are three different keys for Tribal), a function it is
(Yeah we'll throw everything in a function later, probably, since it's just tidy, but this requires it)

In [14]:
# Create a spatial dataframe from the geometries layer
# I really only need the 
geometries_sdf = geometries_layer.query().sdf

c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'services9.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'services9.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'services9.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/l

In [15]:
state = "State or Equivalent"
county = "County or Equivalent"
tribal = "Tribal or Equivalent" 

def get_geometries(entity, field):

    # For the loop I want to make sure I'm only looking at the adds
    # that correspond to the entity level I'm interested in here
    entity_adds_df = adds_df[adds_df["Entity"] == entity]

    merged = entity_adds_df.merge(geometries_sdf, left_on=[key_fields_dict[field]], right_on=field)

    geom_dict = merged.groupby("femaDeclarationString")["SHAPE"].apply(list).to_dict()

    print(f"{entity}: {geom_dict}")


state_geometries = get_geometries(state, state_field)

county_geometries = get_geometries(county, county_field)

tribal1_geometries = get_geometries(tribal, tribal_field1)

tribal2_geometries = get_geometries(tribal, tribal_field2)

tribal3_geometries = get_geometries(tribal, tribal_field3)

State or Equivalent: {}
County or Equivalent: {'FM-5610-OR': [{'rings': [[[-13481100.5781, 5486843.1441], [-13481082.433, 5486842.2134], [-13479949.2004, 5486785.1318], [-13478190.6861, 5486697.1835], [-13475293.7072, 5486552.3109], [-13472816.7367, 5486428.5352], [-13472707.087, 5486423.1064], [-13471892.4509, 5486386.0361], [-13471214.2924, 5486355.1701], [-13470488.7118, 5486322.1328], [-13468242.9521, 5486207.3561], [-13468221.2435, 5476945.8677], [-13468213.0051, 5470965.9548], [-13468214.6748, 5470548.8106], [-13468213.7842, 5469778.9862], [-13468205.1005, 5464151.1038], [-13468195.7495, 5462861.8246], [-13468166.8059, 5459386.7933], [-13455024.4247, 5458970.3312], [-13454589.4993, 5458963.8362], [-13453349.1774, 5458948.0628], [-13444989.5275, 5458841.9795], [-13443981.6407, 5458829.299], [-13443619.8523, 5458829.1444], [-13443579.1093, 5458829.2991], [-13442755.011, 5458833.4745], [-13442043.902, 5458837.0313], [-13441681.0004, 5458838.7324], [-13441635.3594, 5458805.9486], [-1

### Coffee Break 9/5

Dang ChatGPT does make coding that much faster. Not like a person isn't looking up syntax either way...but without Chat you need to wade through _close but not quite_ pages from StackOverflow etc....with Chat you can describe exactly what you're trying to do and not have to piece together info from 5 different examples that took you an hour to assemble.

Okay, well, despite a super-slow start and being super-frustrated and possibly having early onset of dementia, we finished what we set out to do this morning. What's next?

* Dissolve the geometries...I'll look into that now
* For later: smash the final geometry layer for each Dec string into the Summaries df (or a derivative of the Summaries df)

You know...back up...I probably don't want my data above in the format I put it in. I probably don't want a dict and the value as a list of strings. I probably want the original un-smashed version. That's what dissolve_boundaries is going to want, I'll bet...okay okay, lemme commit before I start making changes...